In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from pathlib import Path
import pandas as pd
from scipy.stats import qmc

In [ ]:
import firecurve as fc
import running as rn
import makefiles as mf
import thermalrestart as tr
from Analytical_solution_Dirichlet import analytical_dirichlet
import Post_Processor as pp

In [ ]:
column_names = ["Analysis", "e0", "rho", "E", "mu", "f_c", "f_t", "w", "h_ch", "h_cc", "eps", "B", "l", "F", "t_h",
                "T_max", "r_cool", "H", "t_end", "t_end_guess", "fine_size", "n_elements", "failure", "failure_time",
                "time_thermo", "time_mech", "time_tot", "stiffness", 
                ]
df = pd.DataFrame(columns=column_names)

In [ ]:
base = 

In [ ]:
t_h_bounds = [1, 60] # Heating duration [min]
B_bounds = [260, 500] # Column width [mm]

l_bounds = [t_h_bounds[0], B_bounds[0]]
u_bounds = [t_h_bounds[1], B_bounds[1]]


sampler = qmc.LatinHypercube(2, seed=42)
sample = qmc.scale(sampler.random(100), l_bounds, u_bounds)
variables = np.rint(sample).astype(int)

In [ ]:
for i in range(len(variables)):
    # Setting up directory
    workdir = Path(base) / f'Simulation_{i+1}'
    name_firecurve = f'fr_{i+1}.fct'
    name_thermalfile = f'thermo_{i+1}.IN'
    name_mechfile = f'COLUMN_{i+1}.IN'

    # ----------- Defining Inputs ------------------

    # Structural Inputs
    B = variables[i, 1]/1000          # Column width [m]
    l = 3.65          # Column length [m]
    F = 322000        # Applied force [N]

    # Fire Inputs 
    T0     = 20.0     # ambient [°C]
    t_h    = variables[i, 0]     # heating duration to peak [min]
    T_max  = round(T0 + 1325 * fc.phi(t_h/60), 2)   # peak gas temperature at t = t_h [°C] (<=1325). Tau is equal to 1. (ISO heating)
    # T_max  = 1150   # peak gas temperature at t = t_h [°C] (<=1325).
    r_cool = 10.4      # linear cooling rate [°C/min]; set None to skip cooling
    dt     = 0.5      # time step [min]
    
    # Material Inputs
    e0 = 0.02            # Imperfection [m]
    rho = 420            # Density (Including moisture) [kg/m^3]
    E = 11.5e9           # Modulus of Elasticity [Pa]
    mu = 0.3             # Poisson ratio [-]
    f_c = 24e6           # Compressive strentgh [Pa]
    f_t = 19.2e6         # Tensile strength [Pa]
    w = 12               # Moisture content
    h_ch = 35            # Conection coefficient heating []
    h_cc = 4             # Convection coefficient cooling []
    eps = 0.8            # Relative emissivity [-]
    struc_material_name = 'WOODEC5'                      # Material module for structural simulation
    struc_material_params = (E, mu, f_c, f_t, 1)       # Material parameters corresponding to material module
    therm_material_name = 'WOODEC5'
    therm_material_params = (rho, w, h_ch, h_cc, eps, 4, 0, 0, 1)
    

    #  Numerical Inputs
    fine_size = mf.compute_fine_size_closest(B, 0.005)  # Mesh size of Thermal analysis
    n_elements, mesh_size = mf.compute_mesh_and_nelements(l, 0.25)
    # ---------------------------------------------

    # Calculations
    q_dist = B*B*rho*10
    M0 = e0 * F
    t_end  = round(analytical_dirichlet(B, rho, T_max, t_h, r_cool, c=1530, k=0.12, plot=False)[0]*1.3, 2) # end time of the curve [min] (>= t_h), analytically determined with dirichlet boundary condition of T_g(t) times two.
    t_end_guess = t_end

    # Firecurve
    t_all, T_all, tau = fc.build_fire_curve(T0=T0, T_max=T_max, t_h=t_h, r_cool=r_cool, t_end=t_end, dt=dt, plot_curve=False)
    H = fc.area_firecurve(T0, T_max, t_h, r_cool, t_end, tau)
    outfile = fc.save_fct(t_all, T_all, filename=name_firecurve, out_dir=workdir)
    
    # # Natural fire Tests inputs
    # tau = values[i]
    # print(tau)
    # T_start = 100
    # T_end = 120
    # therm_material_name = 'USER1'
    # therm_material_params = (rho, w, T_start, T_end, h_ch, h_cc, eps, 1, tau)
    # data = np.loadtxt('Natural_fire_test_data/TEST12/TF_12.fct')
    # times = data[:, 0]/60
    # Temps = data[:, 1]
    # t_end = times[-1]
    # outfile = fc.save_fct(times, Temps, filename=name_firecurve, out_dir=workdir)
    
    # Thermal Analysis
    nfiber = mf.make_cross_section_SYM(workdir=workdir, filename=name_thermalfile, B=B, fine_size=fine_size, frontier_name=name_firecurve, T_initial=20.0, comeback=0.001, precision=1e-3, dt0=7.5, t_final=t_end*60, dtmax=30.0, timeprint=60.0, material_name=therm_material_name, material_params=therm_material_params)
    inp = Path(workdir) / Path(name_thermalfile).stem
    rn.run_safir(input_file=inp, env_var="SAFIREXE", workdir=workdir)
    restart = tr.thermal_restart(workdir, name_thermalfile, B, fine_size)
    for r in range(1, 8):
        if restart['runs'] > 1:
            t_end = restart['last_time']/60
            workdir = Path(workdir) / f'Batch0{r}'
            t_all, T_all, tau = fc.build_fire_curve(T0=T0, T_max=T_max, t_h=t_h, r_cool=r_cool, t_end=t_end, dt=dt, plot_curve=False)
            H = fc.area_firecurve(T0, T_max, t_h, r_cool, t_end, tau)
            outfile = fc.save_fct(t_all, T_all, filename=name_firecurve, out_dir=workdir)
            nfiber = mf.make_cross_section_SYM(workdir=workdir, filename=name_thermalfile, B=B, fine_size=fine_size, frontier_name=name_firecurve, T_initial=20.0, comeback=0.01, precision=1e-3, dt0=7.5, t_final=t_end*60, dtmax=30.0, timeprint=60.0, material_name=therm_material_name, material_params=therm_material_params)
            inp = Path(workdir) / Path(name_thermalfile).stem
            rn.run_safir(input_file=inp, env_var="SAFIREXE", workdir=workdir)
            restart = tr.thermal_restart(workdir, name_thermalfile, B, fine_size)
        else:
            break

    
    # Structural Analysis
    tem_file = f"{Path(name_thermalfile).stem}.TEM"
    mf.make_column(workdir=workdir, filename=name_mechfile, length=l, tem_file=tem_file, nfiber=nfiber, elements=n_elements, ng=3 ,comeback=1e-09,  F_function='FLOAD', nodeload1=(0.0, 0.0, -M0), nodeload2=(0.0, -F, M0), q_dist=-q_dist, t_final=t_end*60, timeprint=60.0 ,material_name=struc_material_name, material_params=struc_material_params, stresses=True)
    inp_2 = Path(workdir) / Path(name_mechfile).stem
    rn.run_safir(input_file=inp_2, env_var="SAFIREXE", workdir=workdir)

    # Post-Processing
    failure_time, failure = pp.postprocess_failure(workdir, f'{Path(name_mechfile).stem}.OUT', t_end)
    time_thermo = pp.get_simulation_time(workdir, f'{Path(name_thermalfile).stem}.OUT')
    time_mech = pp.get_simulation_time(workdir, f'{Path(name_mechfile).stem}.OUT')
    time_tot = pp.sum_times([time_thermo, time_mech])
    pp.make_description(workdir, 'AA_Description.txt', time_thermo, time_mech, time_tot, e0, rho, E, mu, f_c, f_t, w, h_ch, h_cc, eps, B, l, F, T0, t_h, T_max, r_cool, t_end, dt, fine_size, n_elements, failure, failure_time)
    stiffness = pp.get_stiffness(workdir, f'{Path(name_mechfile).stem}.OUT', n_elements=n_elements)
    df.loc[i] = [i+1, e0, rho, E, mu, f_c, f_t, w, h_ch, h_cc, eps, B, l, F, t_h, T_max, r_cool, H, t_end, t_end_guess, fine_size, n_elements, failure, failure_time, time_thermo, time_mech, time_tot, stiffness]
    # nodes = []
    # thermo_couples = pp.get_temperatures(workdir, f'{Path(name_thermalfile).stem}.OUT', nodes)
    data_name = f'Data.csv'
    csv = Path(base) / data_name
    df.to_csv(csv, index=False)